In [41]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
sns.set()
from patsy import dmatrix
from taxi_pakage import get_features, erase_outlier
import datetime as dt

In [2]:
taxi = pd.read_csv('edited_taxi.csv')

In [18]:
feature_n, features=get_features(taxi)

In [19]:
test = dmatrix("{}".format(features), taxi, return_type ="dataframe" )
test.tail()

,Intercept,id,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,dist,pick_date,pick_time,drop_time,holiday,holiday_Fri,weekday,trip_duration
1458639,1.0,2376096.0,2.0,4.0,-73.982201,40.745522,-73.994911,40.740170,0.0,1.224311,98.0,48663.0,49441.0,0.0,1.0,4.0,778.0
1458640,1.0,1049543.0,1.0,1.0,-74.000946,40.747379,-73.970184,40.796547,0.0,6.046037,9.0,27314.0,27969.0,1.0,1.0,6.0,655.0
1458641,1.0,2304944.0,2.0,1.0,-73.959129,40.768799,-74.004433,40.707371,0.0,7.819693,112.0,25060.0,25824.0,0.0,1.0,4.0,764.0
1458642,1.0,2714485.0,1.0,1.0,-73.982079,40.749062,-73.974632,40.757107,0.0,1.091878,4.0,57385.0,57758.0,0.0,0.0,1.0,373.0
1458643,1.0,1209952.0,1.0,1.0,-73.979538,40.781750,-73.972809,40.790585,0.0,1.133330,95.0,53064.0,53262.0,0.0,0.0,1.0,198.0


In [20]:
real = test.loc[:, "trip_duration"] # 실제 정답 y
norm = []

In [22]:
feature_n, features=get_features(test, 1, -1)
feature_n, features

(15,
 'id + vendor_id + passenger_count + pickup_longitude + pickup_latitude + dropoff_longitude + dropoff_latitude + store_and_fwd_flag + dist + pick_date + pick_time + drop_time + holiday + holiday_Fri + weekday')

In [23]:
model =  sm.OLS.from_formula("trip_duration ~ {}".format(features), data=test)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     951.8
Date:                Wed, 07 Mar 2018   Prob (F-statistic):               0.00
Time:                        15:22:02   Log-Likelihood:            -1.4554e+07
No. Observations:             1458644   AIC:                         2.911e+07
Df Residuals:                 1458628   BIC:                         2.911e+07
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           1.029e+05   8493.174     12.121      0.000    8.63e+04     1.2e+05
id                  -8.17e-06   3.74e-06     -2.186      0.029   -1.55e-05   -8.45e-07
vendor_id            199.7325      9.061     22.042      0.000     181.972     217.493
passenger_count        8.4738      3.430      2.470      0.013       1.751      15.197
pickup_longitude     -24.7572    101.890     -0.243      0.808    -224.458     174.943
pickup_latitude    -1387.7927    154.266     -8.996      0.000   -1690.148   -1085.437
dropoff_longitude    315.2532    100.092      3.150      0.002     119.077     511.430
dropoff_latitude    -607.7434    139.445     -4.358      0.000    -881.051    -334.436
store_and_fwd_flag    47.6592     58.481      0.815      0.415     -66.962     162.280
dist                 112.1010      1.076    104.211      0.000     109.993     114.209
pick_date              0.4799      0.084      5.718      0.000       0.315       0.644
pick_time             -0.0047      0.001     -9.078      0.000      -0.006      -0.004
drop_time              0.0061      0.001     11.968      0.000       0.005       0.007
holiday             -111.8358     14.647     -7.635      0.000    -140.544     -83.127
holiday_Fri          -25.1899     16.105     -1.564      0.118     -56.756       6.376
weekday               21.5922      4.047      5.335      0.000      13.660      29.525
================================================================================
Omnibus:                  8314824.047   Durbin-Watson:                     2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   2325036240968843.500
Skew:                         347.762   Prob(JB):                           0.00
Kurtosis:                  195591.395   Cond. No.                       4.55e+09
================================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.55e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

1. id 값이 2.9%의 유의확률을 보인다.    
    - a unique identifier for each trip
    - 의미 없는 수치가 아닐까?
    - -> 제거
1. 좌표값 4개중 1개만 유의확률이 확연하게 높다. 
    - 한개를 빼려면 나머지도 다 빼는게 나은가? 
    - 좌표값 4개의 정보는 dist에 포함되어 있으니 제거하는게 마땅한가?
    - -> 제거

1. store_and_fwd_flag 
    - 41% 유의확률- > 제거

1. 휴일에 금요일을 포함하면 유의확률이 11%로 증가한다. 
    - 금요일이 포함되지 않은 feature로 선택한다.

In [24]:
test = test.drop(["id", "dropoff_longitude", "dropoff_latitude", "pickup_longitude", "pickup_latitude","store_and_fwd_flag", "holiday_Fri"], axis = 1)
test.tail()

,Intercept,vendor_id,passenger_count,dist,pick_date,pick_time,drop_time,holiday,weekday,trip_duration
1458639,1.0,2.0,4.0,1.224311,98.0,48663.0,49441.0,0.0,4.0,778.0
1458640,1.0,1.0,1.0,6.046037,9.0,27314.0,27969.0,1.0,6.0,655.0
1458641,1.0,2.0,1.0,7.819693,112.0,25060.0,25824.0,0.0,4.0,764.0
1458642,1.0,1.0,1.0,1.091878,4.0,57385.0,57758.0,0.0,1.0,373.0
1458643,1.0,1.0,1.0,1.133330,95.0,53064.0,53262.0,0.0,1.0,198.0


In [29]:
feature_n, feature = get_features(test, 1, -1)
feature_n, feature

(8,
 'vendor_id + passenger_count + dist + pick_date + pick_time + drop_time + holiday + weekday')

In [30]:
model =  sm.OLS.from_formula("trip_duration ~  {}".format(feature), data=test)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1761.
Date:                Wed, 07 Mar 2018   Prob (F-statistic):               0.00
Time:                        15:22:49   Log-Likelihood:            -1.4554e+07
No. Observations:             1458644   AIC:                         2.911e+07
Df Residuals:                 1458635   BIC:                         2.911e+07
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         110.1788     20.530      5.367      0.000      69.941     150.416
vendor_id         198.5211      9.035     21.974      0.000     180.814     216.229
passenger_count     8.5797      3.430      2.501      0.012       1.856      15.303
dist              115.6586      1.006    115.011      0.000     113.688     117.630
pick_date           0.4853      0.084      5.785      0.000       0.321       0.650
pick_time          -0.0046      0.001     -8.954      0.000      -0.006      -0.004
drop_time           0.0060      0.001     11.761      0.000       0.005       0.007
holiday          -117.1332     13.670     -8.569      0.000    -143.925     -90.341
weekday            18.0796      3.230      5.598      0.000      11.749      24.410
================================================================================
Omnibus:                  8314513.481   Durbin-Watson:                     2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   2324268303439564.500
Skew:                         347.713   Prob(JB):                           0.00
Kurtosis:                  195559.092   Cond. No.                       3.91e+05
================================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.91e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

1. passenger_count 1.2% 유의확률을 보인다. 삭제해야 할까?

1. 레버리지가 큰 값들을 지우고 나면 유의확률이 낮아질까?

In [31]:
test = erase_outlier(result, test, feature_n)
len(test)

2243


1456401

In [37]:
feature_n, feature = get_features(test, 1, -1)
feature_n, feature

(8,
 'vendor_id + passenger_count + dist + pick_date + pick_time + drop_time + holiday + weekday')

In [38]:
model =  sm.OLS.from_formula("trip_duration ~ + {}".format(feature), data=test)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.603
Model:                            OLS   Adj. R-squared:                  0.603
Method:                 Least Squares   F-statistic:                 2.762e+05
Date:                Wed, 07 Mar 2018   Prob (F-statistic):               0.00
Time:                        15:24:40   Log-Likelihood:            -1.0844e+07
No. Observations:             1456401   AIC:                         2.169e+07
Df Residuals:                 1456392   BIC:                         2.169e+07
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         283.0621      1.638    172.837      0.000     279.852     286.272
vendor_id          -0.9338      0.719     -1.299      0.194      -2.343       0.475
passenger_count     4.1769      0.273     15.292      0.000       3.642       4.712
dist              129.2899      0.088   1474.022      0.000     129.118     129.462
pick_date           0.5360      0.007     80.276      0.000       0.523       0.549
pick_time          -0.0017   4.13e-05    -41.478      0.000      -0.002      -0.002
drop_time           0.0029   4.08e-05     71.873      0.000       0.003       0.003
holiday          -143.9388      1.088   -132.311      0.000    -146.071    -141.807
weekday            12.6813      0.257     49.339      0.000      12.178      13.185
==============================================================================
Omnibus:                   769631.914   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         20853291.805
Skew:                           2.008   Prob(JB):                         0.00
Kurtosis:                      21.097   Cond. No.                     3.91e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.91e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

---

#### vendor_id, weekday, holiday는 카테고리 값으로 넣는 것이 옳은가?

In [ ]:
model =  sm.OLS.from_formula("trip_duration ~ C(vendor_id) +C(weekday) + C(holiday) + {} + 0".format(features), data=test)
result = model.fit()
result.summary()

---

In [ ]:
X = dmatrix("{}".format(features), taxi, return_type ="dataframe" )
predictions = result.predict(X) # 실제 데이터를 넣어 모형평가
norm = np.append(norm, np.linalg.norm(predictions-real)/len(X))
plt.scatter(real, predictions)
plt.show()

In [ ]:
test1 = test.loc[0:700000, :]